## **BITS F407 - ARTIFICIAL INTELLIGENCE**
--------------------------------------------------------------------------------


**PROJECT TITLE: ENHANCING TEXT-PATTERN SEARCH IN SQL DATABASES**
--------------------------------------------------------------------------------

***Team number: 68***


---
**Full names of all students in the team: ANIRUDH BAGALKOTKER, SAKAR HIRDE**

---
**Id number of all students in the team: 2021A7PS2682H, 2021A3PS3203H**


## ***1. Import Dependencies***

In [ ]:
%pip install pandas numpy matplotlib mysql-connector

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector
import time
import math

## ***2. Connect to MySQL Database***

In [ ]:
mysqlConnector = mysql.connector.connect(
	host="localhost",
	user="root",
	password="root"
)

print(mysqlConnector)

mysqlCursor = mysqlConnector.cursor()

mysqlCursor.execute("SHOW DATABASES")

for database in mysqlCursor:
	print(database)

## ***3. Create Database***

In [ ]:
mysqlCursor.execute("CREATE DATABASE IF NOT EXISTS AI_PROJ")

mysqlCursor.execute("USE AI_PROJ")

## ***4. Create Tables***

In [ ]:
mysqlCursor.execute("CREATE TABLE IF NOT EXISTS `BOOKS` ( `BID` int(10) UNSIGNED NOT NULL, `ISBN` bigint(13) UNSIGNED DEFAULT NULL, `NAME` varchar(200) COLLATE utf8_unicode_ci NOT NULL, `MRP` int(6) UNSIGNED DEFAULT NULL, `DESCRIPTION` varchar(1000) COLLATE utf8_unicode_ci DEFAULT NULL, `IMG` varchar(3000) COLLATE utf8_unicode_ci DEFAULT NULL, `AUTHOR` varchar(100) COLLATE utf8_unicode_ci DEFAULT NULL, `FORMAT` varchar(25) COLLATE utf8_unicode_ci DEFAULT 'PAPERBACK' COMMENT 'PAPERBACK OR HARDCOVER', `PAGES` smallint(4) DEFAULT NULL, `WEIGHT` smallint(5) DEFAULT NULL, `REVIEW` tinyint(1) UNSIGNED NOT NULL DEFAULT '0') ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;")

mysqlCursor.execute("CREATE TABLE IF NOT EXISTS `INVENTORY` ( `PID` bigint(20) UNSIGNED NOT NULL, `BID` int(10) UNSIGNED NOT NULL, `SID` int(10) UNSIGNED NOT NULL, `COND` varchar(25) COLLATE utf8_unicode_ci NOT NULL DEFAULT 'NEW', `QTY` smallint(5) UNSIGNED NOT NULL DEFAULT '1', `CP` float(7,2) UNSIGNED NOT NULL, `SP` float(7,2) UNSIGNED NOT NULL, `DISCOUNT` float(3,1) UNSIGNED DEFAULT '0.0', `LANG` varchar(25) COLLATE utf8_unicode_ci DEFAULT 'ENGLISH', `PI` int(10) UNSIGNED NOT NULL DEFAULT '0', `CREATED` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP, `MODIFIED` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci;")

We have Already Added the Tables through the MySQL Workbench (Using SQL) and we have generated some data and filled the data in Tables

## ***5. SQL In-Built Functions Query***

There are 4 SQL In-Built Functions Queries

1. **LIKE**
2. **SIMILAR TO**
3. **CONTAINS**
4. **MATCH**

Lets Apply LIKE and MATCH Queries and check the time taken for executing each of the query

In [ ]:
timeArr = []

In [ ]:
def MatchQuery(searchString):
    start_time = time.time()
    sql = """
    SELECT DISTINCT 
        I.PID, I.BID, I.SID, I.COND, I.QTY, I.CP, I.SP, I.DISCOUNT, I.LANG, B.ISBN, B.NAME, 
        B.MRP, B.DESCRIPTION, B.IMG, B.AUTHOR, B.FORMAT, B.PAGES, B.WEIGHT, B.REVIEW, 
    MATCH(B.NAME, B.AUTHOR, B.DESCRIPTION) AGAINST (%s IN BOOLEAN MODE) 
    AS relevance FROM INVENTORY I JOIN BOOKS B ON I.BID = B.BID 
    WHERE MATCH(B.NAME, B.AUTHOR, B.DESCRIPTION) AGAINST (%s IN BOOLEAN MODE) 
    AND (B.NAME LIKE %s OR B.AUTHOR LIKE %s) ORDER BY relevance DESC 
    """
    mysqlCursor.execute(sql, (searchString, searchString, f"%{searchString}%", f"%{searchString}%"))
    for result in mysqlCursor:
        print(result)
    
    time.sleep(5)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for MatchQuery: {elapsed_time} seconds")
    timeArr.append(elapsed_time)

def LikeQuery(searchString):
    start_time = time.time()
    sql = """
        SELECT DISTINCT I.PID, I.BID, I.SID, I.COND, I.QTY, I.CP, I.SP, I.DISCOUNT, I.LANG, 
            B.ISBN, B.NAME, B.MRP, B.DESCRIPTION, B.IMG, B.AUTHOR, B.FORMAT, 
            B.PAGES, B.WEIGHT, B.REVIEW
        FROM INVENTORY I
        JOIN BOOKS B ON I.BID = B.BID
        WHERE (B.NAME LIKE %s OR B.AUTHOR LIKE %s OR B.DESCRIPTION LIKE %s)
    """
    mysqlCursor.execute(sql, (f"%{searchString}%", f"%{searchString}%", f"%{searchString}%"))

    for result in mysqlCursor:
        print(result)
    
    time.sleep(5)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for MatchQuery: {elapsed_time} seconds")
    timeArr.append(elapsed_time)

search = "Rich"
MatchQuery(search)
LikeQuery(search)

## ***6. AI Search Algorithms***

#### **6.1 Importing the Tables into a DataFrame**

In [ ]:
mysqlCursor.execute("SELECT * FROM BOOKS")
books_data = mysqlCursor.fetchall()

# Create a pandas dataFrame for the 'BOOKS' table
books_df = pd.DataFrame(books_data, columns=["BID", "ISBN", "NAME", "MRP", "DESCRIPTION", "IMG", "AUTHOR", "FORMAT", "PAGES", "WEIGHT", "REVIEW"])

# Display the 'BOOKS' dataFrame
print("BOOKS Table:")
print(books_df)

# Execute a SELECT query on the 'INVENTORY' table
mysqlCursor.execute("SELECT * FROM INVENTORY")
inventory_data = mysqlCursor.fetchall()

# Create a pandas dataFrame for the 'INVENTORY' table
inventory_df = pd.DataFrame(inventory_data, columns=["PID", "BID", "SID", "COND", "QTY", "CP", "SP", "DISCOUNT", "LANG", "PI", "CREATED", "MODIFIED"])

# Display the 'INVENTORY' dataFrame
print("\nINVENTORY Table:")
print(inventory_df)

#### **6.2 Fuzzy Search (Levenshtein Distance)**

Fuzzy search techniques assist in locating outcomes that roughly match
the search query. Soundex and Levenshtein distance are two common methods.

We will use the Levenshtein distance function to calculate the distance
between two strings.

In [ ]:
import re

search_term = "Rich"

# Filtered dataframes
filtered_inventory_df = pd.DataFrame()
filtered_books_df = pd.DataFrame()

for col in ['COND', 'LANG']:
    mask = inventory_df[col].str.contains(re.escape(search_term), case=False)
    filtered_inventory_df = pd.concat([filtered_inventory_df, inventory_df[mask]])

for col in ['NAME', 'AUTHOR', 'DESCRIPTION']:
    mask = books_df[col].str.contains(re.escape(search_term), case=False)
    filtered_books_df = pd.concat([filtered_books_df, books_df[mask]])

print("Filtered INVENTORY Table:")
print(filtered_inventory_df)

print("\nFiltered BOOKS Table:")
print(filtered_books_df)

#### **6.3 Vector Space Model (TF-IDF):**

TF-IDF is a statistical tool for assessing a word's significance in relation to a group of documents. It is frequently applied to text-based searches.

In [ ]:
# Define the tfidf function
def tfidf(term, document, documents):
    tf = document.count(term.lower()) / len(document)
    idf = math.log(len(documents) / (1 + sum([1 for doc in documents if term.lower() in doc.lower()])))
    return tf * idf

# Example Usage
search_term = "Rich"

# Apply TF-IDF to relevant columns in inventory_df
for col in ['COND', 'LANG']:
    documents = inventory_df[col].apply(str.split).tolist()
    scores = [sum(tfidf(search_term, doc, documents) for doc in documents)]
    inventory_df[f'{col}_tfidf_score'] = scores * len(inventory_df)  # Repeat the single score for each row

# Apply TF-IDF to relevant columns in books_df
for col in ['NAME', 'AUTHOR', 'DESCRIPTION']:
    documents = books_df[col].apply(str.split).tolist()
    scores = [sum(tfidf(search_term, doc, documents) for doc in documents)]
    books_df[f'{col}_tfidf_score'] = scores * len(books_df)  # Repeat the single score for each row

# Display the dataframes with TF-IDF scores
print("INVENTORY Table with TF-IDF scores:")
print(inventory_df)

print("\nBOOKS Table with TF-IDF scores:")
print(books_df)

#### **6.4 BM25:**

BM25 is an enhanced version of TF-IDF that takes into account things like term
saturation and normalization of document length. It is well known for being efficient at retrieving information.

In [ ]:
def bm25(term, document, documents, k=1.5, b=0.75):
    tf = document.count(term)
    idf = math.log((len(documents) - sum([1 for doc in documents if term in doc]) + 0.5) / (sum([1 for doc in documents if term in doc]) + 0.5) + 1)
    avgdl = sum(len(doc) for doc in documents) / len(documents)
    score = idf * (tf * (k + 1)) / (tf + k * (1 - b + b * len(document) / avgdl))
    return score

# Example Usage
search_term = 'Rich'

# Apply BM25 to relevant columns in inventory_df
start_time = time.time()
for col in ['COND', 'LANG']:
    documents = inventory_df[col].apply(lambda x: str(x).split() if pd.notnull(x) else []).tolist()
    scores = [bm25(search_term, doc, documents) for doc in documents]
    inventory_df[f'{col}_bm25_score'] = scores

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for MatchQuery: {elapsed_time} seconds")

start_time = time.time()
# Apply BM25 to relevant columns in books_df
for col in ['NAME', 'AUTHOR', 'DESCRIPTION']:
    documents = books_df[col].apply(lambda x: str(x).split() if pd.notnull(x) else []).tolist()
    scores = [bm25(search_term, doc, documents) for doc in documents]
    books_df[f'{col}_bm25_score'] = scores

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for MatchQuery: {elapsed_time} seconds")
timeArr.append(elapsed_time)

# Display the dataframes with BM25 scores
print("INVENTORY Table with BM25 scores:")
print(inventory_df)

print("\nBOOKS Table with BM25 scores:")
print(books_df)